# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [8]:
!cat publications.tsv




11/1/18	The limited capacity model of motivated mediated message processing: looking to the future	Annals of the International Communication Association	"In a companion piece (Fisher, Keene, Huskey, & Weber, 2018), we reviewed the foundations and current state of the Limited Capacity Model of Motivated Mediated Message Processing (LC4MP). In this manuscript we return to the three areas investigated in our review: cognitive load, motivation, and memory. In each domain, we: (a) outline areas in which the LC4MP has produced unexpected or ambiguous findings; (b) look broadly at literature from biology, cognitive psychology, and neuroscience to inform and clarify definitions of key terms; (c) develop an updated, cohesive framework of assumptions and predictions of the LC4MP; and (d) propose a roadmap for testing the proposed framework. We conclude with a discussion of the LC4MP's continued relevance for understanding dynamic, interactive, multimodal communication phenomena."	"Fisher, J. 

## Import pandas

We are using the very handy pandas library for dataframes.

In [9]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [10]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,1/1/18,Taking messages into the magnet: Method-theory...,Communication Monographs,Brain imaging techniques within communication ...,"Weber, R., Fisher, J.T. , Hopp, F., & Lonerga...",msgs-into-the-magnet,https://www.tandfonline.com/doi/abs/10.1080/03...
1,11/1/18,The limited capacity model of motivated mediat...,Annals of the International Communication Asso...,"In the 15 years since its inception, the Limit...","Fisher, J. T., Keene, J. R., Huskey, R., & Web...",LC4MP1,https://www.tandfonline.com/doi/full/10.1080/2...
2,11/1/18,The limited capacity model of motivated mediat...,Annals of the International Communication Asso...,"In a companion piece (Fisher, Keene, Huskey, &...","Fisher, J. T., Keene, J. R., Huskey, R., & Web...",LC4MP2,https://www.tandfonline.com/doi/full/10.1080/2...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [11]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [12]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [13]:
!ls ../_publications/

18-LC4MP1.md               18-msgs-into-the-magnet.md
18-LC4MP2.md


In [14]:
!cat ../_publications/18-msgs-into-the-magnet.md

---
title: "Taking messages into the magnet: Method-theory synergy in communication neuroscience"
collection: publications
permalink: /publication/1/1/18-msgs-into-the-magnet
excerpt: 'Brain imaging techniques within communication research have rapidly expanded in popularity in recent years, driven by an increase in access to functional magnetic resonance imaging (fMRI) technology and by theoretical developments within the field. In this manuscript, we present an overview of research from within communication and cognate disciplines that has leveraged insights from fMRI research to &quot;push the envelope,&quot; demonstrating a synergy between methodological and theoretical progress. In addition, we provide a review of fMRI technology, methodology, and theoretical considerations, focusing on recent developments in the cognitive and brain sciences that are of special relevance to communication scholars. Finally, we provide a series of practical recommendations and resources for communic